In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import cv2

print("open cv2 version: %s" % cv2.__version__)

open cv2 version: 3.3.1


In [2]:
def load_imgs(vidPath, firstFrame, lastFrame):
    '''
    Reads frames from video and stores as a list
    
    Parameters: 
    vidPath (string): Path to video
    
    Returns: 
    list of images
    '''
    
    cap = cv2.VideoCapture(vidPath)
    length = np.arange(start, stop)
    print(len(length)) 
    imgs = []
    for ff in length:
        cap.set(1,ff)
        ret, frame = cap.read()
        if np.mod(ff, 100) == 0:
            print(ff) # prints progress in 50 frames

            # convert to grey
            #img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        imgs.append(frame)
#         print("length of imgs: %s" % len(imgs))
            
    cap.release()
    return imgs

In [3]:
# adjust gamma, if your vid is too dark
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [4]:
def plottracks(ax, x_range, y, frameNum, prob_frame, trackColor):
    ax.plot(x_range, y, color = trackColor)
#     ax.plot(frameNum*1/100, prob_frame, 'o',
#                 color = 'k')
#     ax.set_ylim(0,5)
    ax.axvline(x = frameNum/100, 
              color = 'k',
              alpha = 0.5)
    ax.set_xlabel('Time (second)')
#     ax.set_ylabel('Relative Radial Position')

In [5]:
# blue_hue = {'top_color': [158, 202, 225], 'bottom_color': [8,81, 156]}
# green_hue = {'top_color': [223, 238, 218], 'bottom_color': [0, 82, 33]}
# multiPink = {'top_color': [201, 148, 199], 'bottom_color': [152, 0, 67]}
# orange = {'top_color': [254, 237, 222], 'bottom_color': [166, 54, 3]}

# color_list = [[152, 0, 67], [166, 54, 3], [8,81, 156]]
# from matplotlib import cm

In [6]:
#give path to videofiles
vidpath = r"G:\My Drive\Mahad\Light Level Experiments\Videos Used for Analysis/"
export_path = r'./dataFolder/Output/Proboscis/TrackedVideos/'

# the Ref provides the start and stop frames
path_frames = glob.glob(r"./dataFolders/Output/Step5_v3/" + '*.csv')

In [16]:
visitnum = ['FirstVisit/', 'Lastvisit/']

for visit in visitnum:
    # get the tracks
    direc = r"./dataFolders/Output/Proboscis/FilteredTracks/" + visit
    AllTracks = glob.glob(direc + "*.csv")
    
    for t in AllTracks:
        a, b, c, d = os.path.basename(t).split('_')
        name = a + '_' + b + '_' + c
        print(name)
        
        # get the proboscis track
        track = pd.read_csv(t)
        track.head()

        tracks = pd.DataFrame({'x':track.x,
                                  'y':track.y})

        # get the video section
        videoPath = vidpath + name + '.mp4'
        frame_ref = [f for f in path_frames if name+ '_' in f][0]
        file = pd.read_csv(frame_ref)
        
        if visit == visitnum[0]:
            row = 0
        else:
            row = -1
        
        select = file.iloc[row, :]
        start = int(select['In_Frame'] + 1)
        stop = int(select['Out_Frame'])
        
        
        # get vid info
        cap = cv2.VideoCapture(videoPath)

        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        print(length, width, height, fps)
        # load images
        imList = load_imgs(videoPath, start, stop)
        cap.release()

        print("length of section you want to read: %s" % int(stop - start))
        print("length of the image list: %s" % len(imList))
        print("length of the probsocis track: %s" % len(tracks))
        
#         lagPoints = 5 # number of points to plot on each frame
#         alphaVals = np.flip(np.linspace(0, 0.2, lagPoints), axis = 0)
        vidLen = len(tracks)
        
        # make directory to store images
        tempImgDirectory = os.path.join(os.path.dirname(r"./dataFolders/"), "tmpImgs")
        if not os.path.exists(tempImgDirectory):
            os.mkdir(tempImgDirectory)
            
            
        #######################################################
        outputImgs = []
        for frameNum in np.arange(0, vidLen):
            
            # adjust gamma
            image = adjust_gamma(imList[frameNum], 1.5)
            tt = tracks.loc[frameNum, :]
            x = tt.x 
            y = tt.y
            
            output = image.copy()
            rad_circle = 5
            color = (0, 255, 0)
            if np.isnan(x) or np.isnan(y):
                continue
            else:
                cv2.circle(output, (int(x),int(y)),
                           rad_circle, 
                           color,
                           -1)
            
            # save img
            cv2.imwrite(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"),
                       output)
#             plt.savefig(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"))
#             plt.close()
            
            if np.mod(frameNum, 100) == 0:
                print(frameNum, " of ", vidLen)
                
        ##################################################################################
        
        # convert images to video with ffmpeg
        os.chdir(tempImgDirectory)
        print(tempImgDirectory)

        # use ffmpeg to convert directory to video
        # -r is output frame rate
        os.system('ffmpeg -start_number 0 -r 20 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y outputVid.mp4')

        nameofVideo = r'./' + name + '_' + visit[:-1] + '.mp4'
        os.rename(r'./outputVid.mp4', nameofVideo)
        os.chdir('../../')

        # delete images from directory
        delFiles = [f for f in os.listdir(tempImgDirectory) if f.endswith("png")]
        dlfs = [os.remove(os.path.join(tempImgDirectory, delFiles[ii])) for ii in range(len(delFiles))]

L0.1_c-3_m10
140688 648 488 30.0
335
20400
20500
20600
20700
length of section you want to read: 335
length of the image list: 335
length of the probsocis track: 273
0  of  273
100  of  273
200  of  273
./dataFolders\tmpImgs
L0.1_c-3_m12
97782 648 488 30.0
1976
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
length of section you want to read: 1976
length of the image list: 1976
length of the probsocis track: 1914
0  of  1914
100  of  1914
200  of  1914
300  of  1914
400  of  1914
500  of  1914
600  of  1914
700  of  1914
800  of  1914
900  of  1914
1000  of  1914
1100  of  1914
1200  of  1914
1300  of  1914
1600  of  1914
1700  of  1914
1800  of  1914
1900  of  1914
./dataFolders\tmpImgs
L0.1_c-3_m20
209925 648 488 30.0
7084
46000
46100
46200
46300
46400
46500
46600
46700
46800
46900
47000
47100
47200
47300
47400
47500
47600
47700
47800
47900
48000
48100
48200
48300
48400
48500
48600
48700
48800
48900
49000
49100


59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
length of section you want to read: 5991
length of the image list: 5991
length of the probsocis track: 571
0  of  571
100  of  571
200  of  571
300  of  571
400  of  571
500  of  571
./dataFolders\tmpImgs
L0.1_c-3_m45
183848 648 488 30.000030188011753
1226
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
length of section you want to read: 1226
length of the image list: 1226
length of the probsocis track: 1129
0  of  1129
100  of  1129
200  of  1129
300  of  1129
500  of  1129
600  of  1129
700  of  1129
800  of  1129
900  of  1129
1000  of  1129
1100  of  1129
./dataFolders\tmpImgs
L0.1_c-3_m46
49883 648 488 30.00003127321184
157
11400
length of section you want to read: 157
length of the image list: 157
length of the probsocis track: 157
0  of  157
100  of  157
./dat

./dataFolders\tmpImgs
L50_c-3_m32
211993 648 488 30.0
698
39700
39800
39900
40000
40100
40200
40300
length of section you want to read: 698
length of the image list: 698
length of the probsocis track: 593
0  of  593
100  of  593
200  of  593
300  of  593
400  of  593
500  of  593
./dataFolders\tmpImgs
L50_c-3_m33
158127 648 488 30.0
359
27000
27100
27200
length of section you want to read: 359
length of the image list: 359
length of the probsocis track: 332
0  of  332
100  of  332
200  of  332
300  of  332
./dataFolders\tmpImgs
L50_c-3_m34
100275 648 488 30.0
1098
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
length of section you want to read: 1098
length of the image list: 1098
length of the probsocis track: 852
0  of  852
100  of  852
200  of  852
300  of  852
600  of  852
800  of  852
./dataFolders\tmpImgs
L50_c-3_m35
120463 648 488 30.0
2412
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
3

./dataFolders\tmpImgs
L0.1_c-3_m32
174311 648 488 30.0
1966
158600
158700
158800
158900
159000
159100
159200
159300
159400
159500
159600
159700
159800
159900
160000
160100
160200
160300
160400
length of section you want to read: 1966
length of the image list: 1966
length of the probsocis track: 1662
0  of  1662
100  of  1662
200  of  1662
300  of  1662
400  of  1662
500  of  1662
600  of  1662
700  of  1662
800  of  1662
900  of  1662
1000  of  1662
1100  of  1662
1200  of  1662
1300  of  1662
1400  of  1662
1500  of  1662
1600  of  1662
./dataFolders\tmpImgs
L0.1_c-3_m34
162968 648 488 30.0
2143
81900
82000
82100
82200
82300
82400
82500
82600
82700
82800
82900
83000
83100
83200
83300
83400
83500
83600
83700
83800
83900
length of section you want to read: 2143
length of the image list: 2143
length of the probsocis track: 186
0  of  186
100  of  186
./dataFolders\tmpImgs
L0.1_c-3_m37
150727 648 488 30.0
680
124600
124700
124800
124900
125000
125100
125200
length of section you want to r

L50_c-3_m14
140125 648 488 30.0
1322
80400
80500
80600
80700
80800
80900
81000
81100
81200
81300
81400
81500
81600
length of section you want to read: 1322
length of the image list: 1322
length of the probsocis track: 476
0  of  476
100  of  476
200  of  476
300  of  476
400  of  476
./dataFolders\tmpImgs
L50_c-3_m15
148664 648 488 30.0
3301
86000
86100
86200
86300
86400
86500
86600
86700
86800
86900
87000
87100
87200
87300
87400
87500
87600
87700
87800
87900
88000
88100
88200
88300
88400
88500
88600
88700
88800
88900
89000
89100
89200
length of section you want to read: 3301
length of the image list: 3301
length of the probsocis track: 1215
0  of  1215
100  of  1215
200  of  1215
300  of  1215
400  of  1215
500  of  1215
600  of  1215
700  of  1215
800  of  1215
900  of  1215
1000  of  1215
1100  of  1215
1200  of  1215
./dataFolders\tmpImgs
L50_c-3_m21
186985 648 488 30.0
4308
146800
146900
147000
147100
147200
147300
147400
147500
147600
147700
147800
147900
148000
148100
148200
148

In [9]:
os.chdir('../../')

In [10]:
os.getcwd()

'C:\\Users\\Tanvi\\Documents\\GitHub\\ManducaMultiSenseFeeding'

In [14]:
np.isnan(x)

True

In [ ]:
outputImgs = []
for frameNum in np.arange(strt, vidLen):

    # adjust gamma
    image = adjust_gamma(imList[frameNum], 1.5)

    # convert to rgb, so I can add colored points
    #image = cv2.cvtColor(im1,cv2.COLOR_GRAY2RGB)

    overlay = image.copy()
    output = image.copy()
    for idx, jj in enumerate(range(lagPoints)):
        
        for tt, cc in zip(tracks, colors):
            x = tt.x 
            y = tt.y
        
            if np.isnan(x[np.max([frameNum-jj, 0])]) or np.isnan(y[np.max([frameNum-jj, 0])]) :
                continue                                          

            # add circle, centered at closest integer value
            cv2.circle(output, (int(x[np.max([frameNum-jj, 0])]), 
                             int(y[np.max([frameNum-jj, 0])])), 
                   3, np.array(cc(idx/lagPoints)[:3])*256, -1)

    f = plt.figure(figsize = (8, 3*2))
    ax0 = f.add_subplot(111)
        
    ax0.imshow(output)
    ax0.set_xticks([])
    ax0.set_yticks([])

#     save img
    plt.savefig(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"))
    plt.close()

    if np.mod(frameNum, 100) == 0:
        print(frameNum, " of ", vidLen)

In [ ]:
# convert images to video with ffmpeg
os.chdir(tempImgDirectory)
print(tempImgDirectory)

# use ffmpeg to convert directory to video
# -r is output frame rate
os.system('ffmpeg -start_number 0 -r 20 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y outputVid.mp4')

nameofVideo = r'./' + mothID    + '_AntennaeHeadProboscis.mp4'
os.rename(r'./outputVid.mp4', nameofVideo)
os.chdir('../../../../../../PaperPinelineCodes/')

# delete images from directory
delFiles = [f for f in os.listdir(tempImgDirectory) if f.endswith("png")]
dlfs = [os.remove(os.path.join(tempImgDirectory, delFiles[ii])) for ii in range(len(delFiles))]

In [ ]:
os.chdir('../../../../../../PaperPinelineCodes/')